# Import all necessary modules

In [2]:
# import all neceassary modules
from Experimentation.monte_carlo import *
from Experimentation.ensemble_models import *
from Experimentation.get_alpha import *
from Experimentation.quantile import *
from Experimentation.weighting_scheme import *
from Experimentation.wis_score import *
from Experimentation.model_averaging import *
import numpy as np
import pandas as pd
import random
random.seed(42)
np.random.seed(42)

In [9]:
filename = 'data/alpha.txt'
alphas = get_alpha(filename)

state = 'Monroe' #if use Monroe or SantaRosa for Monroe or Santa Rosa county

if state=='New York':
    s_id = '36'
elif state=='Monroe':
    s_id='Monroe'
elif state=='Santa Rosa':
    s_id='SantaRosa'

# Ensemble Weights Calculation

Weights are calculated based on the formula used in paper.

In [11]:
fname = 'data/all_methods_state_{}.csv'.format(s_id)
data = pd.read_csv(fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
data = model_weights(data, t_p=4)
data = change_weights_format(data)
data.to_csv('weights_'+fname.split('/')[-1]) ## creates a new file containing weights

1
1
1
1
1
1
1
1


# Ensemble Forecast using LOP, Vincent, Mean, Median averaging

In [8]:
fname = 'weights_all_methods_state_{}.csv'.format(s_id)
weights = pd.read_csv(fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
f_fname = 'data/all_methods_state_{}.csv'.format(s_id)
data = pd.read_csv(f_fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
out = LOP_Ensemble(data, weights, alphas)  ## for vincent use Vincent_Ensemble, for mean use Mean_Ensemble, and for median use Median_Ensemble
out.to_csv('LOP-Ensemble_{}.csv'.format(s_id))

# Calculate WIS

In [ ]:
fname = 'LOP-Ensemble_{}.csv'.format(s_id) #filename to get WIS score
data = pd.read_csv(fname, dtype={'location':str,'FIPS':str}, parse_dates=['target_end_date','forecast_date'], index_col=0)
data = get_all_wis_score(data)
data.to_csv(fname)